In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df = spark.sql("select * from databricks_catalog.silver.customers_silver")

In [0]:
df.display()

In [0]:
df = df.dropDuplicates(subset=['customer_id'])

In [0]:
df.display()

In [0]:
df = df.withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))



In [0]:
df.display()

In [0]:

init_load_flag=0


In [0]:
if init_load_flag == 0:
    df_old = spark.sql("select DimCustomerKey, customer_id,create_date, update_date from databricks_catalog.gold.DimCustomers")
else : 
    df_old = spark.sql('''select 0 DimCustomerKey, 0 customer_id, 0 create_date, 0 update_date from databricks_catalog.silver.customers_silver where 1=0''')


In [0]:
df_old = df_old.withColumnRenamed("DimCustomerKey", "old_DimCustomerKey").withColumnRenamed("customer_id", "old_customer_id").withColumnRenamed("create_date", "old_create_date").withColumnRenamed("update_date", "old_update_date")

In [0]:
df_old.display()

In [0]:
df_join = df.join(df_old, df.customer_id == df_old.old_customer_id, how='left')
df_join.display()

In [0]:
df_new = df_join.filter(df_join.DimCustomerKey.isNull())
df_new.display()

In [0]:
df_old = df_join.filter(df_join.DimCustomerKey.isNotNull())
df_old.display()

In [0]:
df_old = df_old.drop("old_DimCustomerKey", "old_customer_id",  "old_update_date")
df_old = df_old.withColumnRenamed("old_create_date", "create_date")
df_old= df_old.withColumn("create_date", to_timestamp(col("create_date")))
df_old = df_old.withColumn("update_date", current_timestamp())
df_old.display()


In [0]:
df_new = df_new.drop("old_DimCustomerKey", "old_customer_id",  "old_create_date", "old_update_date")
df_new = df_new.withColumn("create_date", current_timestamp())
df_new = df_new.withColumn("update_date", current_timestamp())
df_new.display()


In [0]:
df_new = df_new.withColumn("DimCustomerKey", monotonically_increasing_id()+lit(1))
df_new.display()


In [0]:
if init_load_flag == 1:
    max_surrogate_key = 0
else : 
    df_maxsur = spark.sql("select max(DimCustomerKey) as max_surrogate_key from databricks_catalog.gold.DimCustomers")
    max_surrogate_key = df_maxsur.collect()[0]['max_surrogate_key']


In [0]:
df_new.display()

In [0]:
df_old.display()

In [0]:
df_new = df_new.withColumn("DimCustomerKey", lit(max_surrogate_key)+col("DimCustomerKey"))


In [0]:
df_new.display()
df_old.display()


In [0]:
df_final = df_new.union(df_old)
df_final.display()


In [0]:
from delta.tables import *


In [0]:
if spark.catalog.tableExists("databricks_catalog.gold.DimCustomers"):
    dlt_obj = DeltaTable.forPath(spark,"abfss://gold@databricksetecharif.dfs.core.windows.net/DimCustomers")
    dlt_obj.alias("trg").merge(df_final.alias("src"), "trg.DimCustomerKey = src.DimCustomerKey").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()
else:
    df_final.write.mode("overwrite").format("delta").option("path","abfss://gold@databricksetecharif.dfs.core.windows.net/DimCustomers").saveAsTable("databricks_catalog.gold.DimCustomers")


In [0]:
%sql
SELECT * from databricks_catalog.gold.dimcustomers